In [21]:
import os
import pandas as pd
import cv2

In [1]:
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [2]:
from keras.models import load_model

Using TensorFlow backend.


In [8]:
os.getcwd()

'c:\\auto_ml'

In [11]:
sorted(os.listdir('keras-retinanet/snapshots'))

['_.txt']

In [13]:
model_path = os.path.join('keras-retinanet/snapshots', sorted(os.listdir('keras-retinanet/snapshots'), reverse=True)[0])
print(model_path)

keras-retinanet/snapshots\_.txt


In [14]:
model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

OSError: Unable to open file (file signature not found)

In [17]:
labels_to_names = pd.read_csv('data/classes.csv', header=None).T.loc[0].to_dict()

In [19]:
labels_to_names

{0: 'hand'}

In [22]:
import skimage.io as io

def predict(image):
    image = preprocess_image(image.copy())
    image, scale = resize_image(image)

    boxes, scores, labels = model.predict_on_batch(
     np.expand_dims(image, axis=0)
     )

    boxes /= scale

    return boxes, scores, labels

# Captura de Video
camera = cv2.VideoCapture(0)
camera_height = 500

while(True):

    _, frame = camera.read()
    

    frame = cv2.flip(frame, 1)

    boxes, scores, labels = predict(frame)

    draw = frame.copy()


    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score > 0.8:
            print(box)
            b = box.astype(int)
            color = label_color(label)
            draw_box(draw, b, color=color)
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)
            

    # show the frame
    cv2.imshow("Test out", draw)


    key = cv2.waitKey(1)

    # quit camera if 'q' key is pressed
    if key & 0xFF == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()

NameError: name 'model' is not defined